# Amazon Bedrock AgentCore Observability: データ保護

複雑なワークフローや意思決定プロセスを自動化するエージェント AI システムを採用する組織が増えるにつれ、機密データの保護が重要な課題となっています。AI エージェントは、入力処理から出力生成まで、エージェントのライフサイクル全体を通じて保護されなければならない個人識別情報（PII）、財務データ、健康記録、その他の機密情報を扱うことが多くあります。

このノートブックでは、Amazon Bedrock Guardrails と Amazon CloudWatch Logs データ保護ポリシーを組み合わせて、エージェント AI アプリケーションの機密データを保護する包括的なアプローチを説明します。このデモンストレーションでは Strands フレームワークを使用して AgentCore Runtime にエージェントをホストしますが、これらのデータ保護の原則と技術は、任意のランタイムや任意のフレームワークでホストされているエージェントに適用でき、概念はプラットフォームに依存せず、既存のエージェントインフラストラクチャに適応可能です。


# 学習内容
このハンズオンチュートリアルでは、以下を探求します:

- エージェントのインタラクション、CloudWatch Logs、およびトレースで機密情報を検出する方法
- Amazon Bedrock Guardrails: AI エージェントが機密コンテンツを処理または生成することを防ぐために機密情報フィルターを設定する方法
- CloudWatch Logs データ保護: アプリケーションログで機密データを自動的に検出してマスクし、PII やその他の機密情報がログメカニズムを通じて漏洩しないようにする方法
- AgentCore 統合: これらの保護対策をエージェントワークフロー内に実装し、AI アプリケーションの多層防御戦略を作成する方法

# なぜこれが重要か

適切な保護がなければ、エージェント AI システムは以下のことが起こり得ます:

- レスポンスやログで機密の顧客データを誤って公開する
- プライバシー規制（GDPR、HIPAA、CCPA）に違反する情報を処理または保持する
- 共有すべきでない PII を含む出力を生成する
- アプリケーションインフラストラクチャにコンプライアンスおよびセキュリティの脆弱性を作成する

Bedrock Guardrails と CloudWatch Logs データ保護を一緒に実装することで、入力から出力、そしてログ記録まで連携して機能する複数の保護層を作成し、エージェント AI アプリケーションを保護できます。


# Amazon Bedrock AgentCore エージェントの可観測性データを表示する

エージェントに可観測性を実装した後、CloudWatch コンソールの生成 AI 可観測性ページと CloudWatch Logs の両方で[収集されたログ、メトリクス、トレースを表示](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-view.html)できます。個々のセッションやトレースデータの確認方法を含む CloudWatch での生成 AI 可観測性の使用方法について詳しくは、Amazon CloudWatch ユーザーガイドの [Amazon Bedrock AgentCore エージェント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AgentCore-Agents.html)を参照してください。

`AgentCore エージェントのロググループは以下の形式です`:

`1. 標準ログ`

- 標準ログ形式: stdout/stderr 出力
- 場所: /aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/[runtime-logs] <UUID>
- 含まれるもの: ランタイムエラー、アプリケーションログ、デバッグ文

使用例:
- print("Processing request...") # 標準ログに表示
- logging.info("Request processed successfully") # 標準ログに表示


`2. OTEL 構造化ログ - 詳細な操作情報`

- 場所: /aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/otel-rt-logs
- 含まれるもの: 実行詳細、エラー追跡、パフォーマンスデータ
- 自動収集: 追加コード不要 - ADOT インストルメンテーションによって生成
- メリット: 関連トレースにログをリンクする相関 ID を含めることができる

`3. トレースとスパン`

トレースは、エージェントを通じたリクエスト実行パスの可視性を提供します:

- 場所: /aws/spans/default
- アクセス方法: CloudWatch Transaction Search コンソール
- 要件: CloudWatch Transaction Search を有効にする必要がある

トレースは以下を自動的にキャプチャします:
- エージェント呼び出しシーケンス
- フレームワークコンポーネント（LangChain など）との統合
- LLM 呼び出しとレスポンス
- ツール呼び出しと結果
- エラーパスと例外


<span style="color:red;">このラボでは、標準ログの保護に焦点を当てます。</span>

# 前提条件

- Amazon CloudWatch で Transaction Search を有効化。初回ユーザーは Bedrock AgentCore のスパンとトレースを表示するために [CloudWatch Transaction Search を有効化](../../00-enable-transaction-search-template/enable_transaction_search.ipynb)する必要があります。
- Claude Haiku 4.5（モデル ID: global.anthropic.claude-haiku-4-5-20251001-v1:0）への Amazon Bedrock モデルアクセスを持つ AWS アカウント
- `aws configure` で AWS 認証情報を設定済み
- データ保護ポリシーの作成または操作に必要な適切な IAM 権限を付与する[ドキュメント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/data-protection-policy-permissions.html)、Bedrock Guardrails および AgentCore


# 1. セットアップと設定

必要な依存関係をインストール:

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet


# 2. データ保護を有効にしないエージェントの作成

まず、データ保護を有効にしないエージェントを実行して、結果を確認しましょう。これには、機密情報を含むコンタクトセンターエージェントと顧客のやり取りのサンプルデータセットを使用します。内容は[こちら](./data/customer_support_conversation_sample.txt)で確認できます。サンプルデータで提供された会話を要約するようにエージェントプロンプトを更新しました。

また、以下のように `print` 文に機密情報を追加しました:

            "agent.type": "customer_agent_reviewer",
            "agent.email": "jrussell@domain.com",
            "agent.phone": "301-555-0100",
            "agent.id": "ABCDE12345"

以下のようにエージェントに機密情報を明らかにするよう意図的にプロンプトしました:
            
            user_input = """summarize the agent conversation. Tell me exactly the phone number, name, address, and email?"""


In [ ]:
%%writefile data_protection.py
import os
import logging
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# ロギングの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Strands ロギングの設定
logging.getLogger("strands").setLevel(logging.INFO)

app = BedrockAgentCoreApp()

@tool
def agent_call_summary(query: str) -> str:
    """コンタクトセンターエージェントのインタラクションを要約します。"""
    try:
        logger.info(f"エージェント通話サマリーを処理中。クエリ: {query[:50]}...")
        
        # カスタマーサポート会話データを読み込み
        results = open('./data/customer_support_conversation_sample.txt', 'r').read()
        
        logger.info(f"エージェント会話検索が正常に完了しました。クエリ: {query[:50]}...")
        return results
        
    except Exception as e:
        logger.error(f"エージェント通話サマリーに失敗しました: {str(e)}")
        return f"検索エラー: {str(e)}"

def get_bedrock_model():
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")
    
    try:
        bedrock_model = BedrockModel(
            model_id=model_id,
            temperature=0.7,
            max_tokens=1028         
        )
        logger.info(f"Bedrock モデルを正常に初期化しました: {model_id}")
        return bedrock_model
    except Exception as e:
        logger.error(f"Bedrock モデルの初期化に失敗しました: {str(e)}")
        logger.error("適切な AWS 認証情報が設定されていることと、Bedrock モデルへのアクセス権があることを確認してください")
        raise

# モデルとエージェントを初期化
bedrock_model = get_bedrock_model()


# カスタマーサポートエージェントを作成
support_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are an expert customer support conversation agent specializing in finding 
                     accurate and relevant information. Your role is to efficiently search, analyze, and synthesize
                     information provided to answer user queries comprehensively. You should provide
                     well-researched responses with current information, clear summaries, and cite reliable sources
                     when presenting your findings.""",
    tools=[agent_call_summary],
    trace_attributes={        
        "tags": ["Strands", "Observability", "CustomerSupport"]
    }
)

@app.entrypoint
def customer_support_agent(payload):
    """ペイロードでカスタマーサポートエージェントを呼び出します"""
    try:
        # ペイロードからユーザー入力を抽出
        user_input = payload.get("prompt", "")
        
        logger.info(f"ユーザー入力: {user_input[:100]}...")

        print("agent.type: customer_agent_reviewer")
        print("agent.email: jrussell@domain.com")
        print("agent.phone: 301-555-0100")
        print("agent.id: ABCDE12345")
        
        # 特定のクエリが指定されていない場合、デフォルトを使用
        if not user_input:
            user_input = "summarize the agent conversation. Tell me exactly the phone number, name, address, and email."
        
        # カスタマーリサーチタスクを実行
        response = support_agent(user_input)
        
        logger.info("エージェントレスポンスが正常に生成されました")
        return response.message['content'][0]['text']
        
    except Exception as e:
        logger.error(f"エージェント実行に失敗しました: {str(e)}")
        return f"リクエスト処理エラー: {str(e)}"

if __name__ == "__main__":
    app.run()


# AgentCore Runtime へのエージェントのデプロイ

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "customer_support_agent"
response = agentcore_runtime.configure(
    entrypoint="data_protection.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    memory_mode='NO_MEMORY'
)
response

# AgentCore Runtime へのエージェントの起動

In [ ]:
launch_result = agentcore_runtime.launch()

# AgentCore Runtime の呼び出し

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "summarize the agent conversation. Tell me exactly the phone number, name, address, and email."})
invoke_response



ここからエージェントのインタラクションの結果を確認しましょう。注意: 生成されるレスポンスは異なる場合がありますが、概念は同じです！


<div style="text-align:left">
    <img src="images/agent_response_without_data_protection.png" width="100%"/>
</div>



CloudWatch トレース:


<div style="text-align:left">
    <img src="images/trace_without_data_protection.png" width="100%"/>
</div>


CloudWatch ログ（エージェントランタイムログ）:


<div style="text-align:left">
    <img src="images/logs_without_data_protection.png" width="100%"/>
</div>




# 3. Bedrock Guardrails の有効化

Amazon Bedrock の Guardrails は、ユースケース固有のポリシーに基づいてユーザー入力と FM レスポンスを評価し、基盤となる FM に関係なく追加のセーフガード層を提供します。Guardrails は、ファインチューニングされたモデルを含む Amazon Bedrock 上のすべての大規模言語モデル（LLM）に適用できます。顧客は、異なるコントロールの組み合わせで設定された複数のガードレールを作成し、異なるアプリケーションやユースケースでこれらのガードレールを使用できます。

Amazon Bedrock Guardrails は、生成 AI アプリケーションを保護するために複数の方法で使用できます。例えば:

- チャットボットアプリケーションでは、有害なユーザー入力や有毒なモデルレスポンスをフィルタリングするためにガードレールを使用できます。
- 銀行アプリケーションでは、投資アドバイスの要求や提供に関連するユーザークエリやモデルレスポンスをブロックするためにガードレールを使用できます。
- ユーザーとエージェント間の会話トランスクリプトを要約するコールセンターアプリケーションでは、ユーザーのプライバシーを保護するためにユーザーの個人識別情報（PII）を編集するためにガードレールを使用できます。

Amazon Bedrock の Guardrails には、コンテンツフィルター、拒否トピック、単語とフレーズフィルター、機密情報（PII、PHI）フィルターなど、複数のコンポーネントがあります。完全なリストは[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html)を確認してください。

この演習では、機密情報の保護にのみ焦点を当てます。

以下のように Bedrock Guardrail を作成します。ガードレールをデモンストレーションするために、機密情報を匿名化しましたが、プロンプト/レスポンスを完全に `BLOCK` することも選択できます。利用可能なすべての機密情報フィルターについては、[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-sensitive-filters.html)を参照してください。



In [ ]:
import boto3
bedrock_client = boto3.client('bedrock')

create_response = bedrock_client.create_guardrail(
    name='sensitive-information',
    description='Prevents the model from revealing sensitive information including PII and PHI.',
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {
                'type': 'EMAIL',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'PHONE',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'NAME',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'US_SOCIAL_SECURITY_NUMBER',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'US_BANK_ACCOUNT_NUMBER',
                'action': 'ANONYMIZE'
            },
            {
                'type': 'CREDIT_DEBIT_CARD_NUMBER',
                'action': 'ANONYMIZE'
            }
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    blockedInputMessaging='Sorry, guardrails intervened and model cannot answer the question.',
    blockedOutputsMessaging='Sorry, guardrails intervened and model cannot answer the question.',
)

print(create_response)
guardrailId = create_response['guardrailId']



エージェントに対して使用するガードレールの新しいバージョンを作成します。

In [ ]:
version_response = bedrock_client.create_guardrail_version(
    guardrailIdentifier=guardrailId,
    description='Version of Guardrail that has HIGH content filters across'
)
guardrail_version_number = version_response['version']

print(guardrail_version_number)
print(version_response)




以下のように新しく作成した Guardrail をエージェントに適用します:

            guardrail_id=os.getenv("BEDROCK_GUARDRAIL_ID"),      # Bedrock guardrail ID
            guardrail_version=os.getenv("BEDROCK_GUARDRAIL_VERSION"),                   # ガードレールバージョン
            guardrail_trace="enabled",               # デバッグ用にトレース情報を有効化 

Strands Agents にガードレールを適用する方法について詳しくは、[ドキュメント](https://strandsagents.com/latest/documentation/docs/user-guide/safety-security/guardrails/)を参照してください。

注意: このラボでは、便宜上、上記で作成したガードレール ID とバージョン番号を使用しました。ただし、必要に応じて特定の guardrailId とバージョンに置き換えることができます。


In [ ]:
%%writefile data_protection.py
import os
import logging
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# ロギングの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Strands ロギングの設定
logging.getLogger("strands").setLevel(logging.INFO)

app = BedrockAgentCoreApp()



@tool
def agent_call_summary(query: str) -> str:
    """コンタクトセンターエージェントのインタラクションを要約します。"""
    try:
        logger.info(f"エージェント通話サマリーを処理中。クエリ: {query[:50]}...")
        
        # カスタマーサポート会話データを読み込み
        results = open('./data/customer_support_conversation_sample.txt', 'r').read()
        
        logger.info(f"エージェント会話検索が正常に完了しました。クエリ: {query[:50]}...")
        return results
        
    except Exception as e:
        logger.error(f"エージェント通話サマリーに失敗しました: {str(e)}")
        return f"検索エラー: {str(e)}"

def get_bedrock_model():
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")
    
    logger.info(f"BEDROCK_GUARDRAIL_ID: {os.getenv('BEDROCK_GUARDRAIL_ID')}")
    logger.info(f"BEDROCK_GUARDRAIL_VERSION: {os.getenv('BEDROCK_GUARDRAIL_VERSION')}")

    
    try:
        bedrock_model = BedrockModel(
            model_id=model_id,
            temperature=0.7,
            max_tokens=1028,
            guardrail_id=os.getenv("BEDROCK_GUARDRAIL_ID"),           # Bedrock ガードレール ID
            guardrail_version=os.getenv("BEDROCK_GUARDRAIL_VERSION"),                        # ガードレールバージョン
            guardrail_trace="enabled",                    # デバッグ用にトレース情報を有効化            
        )        
        logger.info(f"Bedrock モデルを正常に初期化しました: {model_id}")
        return bedrock_model
    except Exception as e:
        logger.error(f"Bedrock モデルの初期化に失敗しました: {str(e)}")
        logger.error("適切な AWS 認証情報が設定されていることと、Bedrock モデルへのアクセス権があることを確認してください")
        raise

# モデルとエージェントを初期化
bedrock_model = get_bedrock_model()

# カスタマーサポートエージェントを作成
support_agent = Agent(
    model=bedrock_model,
    system_prompt="""You are an expert customer support conversation agent specializing in finding 
                     accurate and relevant information. Your role is to efficiently search, analyze, and synthesize
                     information provided to answer user queries comprehensively. You should provide
                     well-researched responses with current information, clear summaries, and cite reliable sources
                     when presenting your findings.""",
    tools=[agent_call_summary],
    trace_attributes={
        "tags": ["Strands", "Observability", "CustomerSupport"]
    }
)

@app.entrypoint
def customer_support_agent(payload):
    """ペイロードでカスタマーサポートエージェントを呼び出します"""
    try:
        # ペイロードからユーザー入力を抽出
        user_input = payload.get("prompt", "")
        
        logger.info(f"ユーザー入力: {user_input[:100]}...")

        print("agent.type: customer_agent_reviewer")
        print("agent.email: jrussell@domain.com")
        print("agent.phone: 301-555-0100")
        print("agent.id: ABCDE12345")
        
        # 特定のクエリが指定されていない場合、デフォルトを使用
        if not user_input:
            user_input = "summarize the agent conversation. Tell me exactly the phone number, name, address, and email."
        
        # カスタマーリサーチタスクを実行
        response = support_agent(user_input)
        
        logger.info("エージェントレスポンスが正常に生成されました")
        return response.message['content'][0]['text']
        
    except Exception as e:
        logger.error(f"エージェント実行に失敗しました: {str(e)}")
        return f"リクエスト処理エラー: {str(e)}"

if __name__ == "__main__":
    app.run()


では、エージェントを再起動しましょう

# AgentCore Runtime へのエージェントの起動

In [ ]:
launch_result

In [ ]:
launch_result = agentcore_runtime.launch(
    env_vars={
        "BEDROCK_GUARDRAIL_ID": guardrailId, 
        "BEDROCK_GUARDRAIL_VERSION": guardrail_version_number  
    }
)
launch_result

# エージェントを再度テスト

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "summarize the agent conversation. Tell me exactly the phone number, name, address, and email."})
invoke_response


Guardrails を適用した結果を確認しましょう。

ガードレールの設定に従って、機密情報が「匿名化」されているのがわかります。注意: ガードレールに `address` を含めなかったため、住所はまだ表示されています。


<div style="text-align:left">
    <img src="images/agent_response_with_bedrock_guardrails.png" width="100%"/>
</div>



以下の CloudWatch トレース情報では、機密情報が匿名化されています（住所を除く - ガードレールに含めませんでした）。

<div style="text-align:left">
    <img src="images/trace_with_bedrock_guardrails.png" width="100%"/>
</div>




以下に示すように、ログ内の機密情報（`print` 文から）はまだ公開されています。


<div style="text-align:left">
    <img src="images/logs_with_guardrails_no_logs_data_protection.png" width="100%"/>
</div>






# 4. CloudWatch Logs データ保護の有効化

Guardrails は、プロンプトとエージェントのレスポンスで機密情報を保護するのに役立ちます。[CloudWatch Logs データ保護](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/mask-sensitive-log-data.html)は、ログ内の機密情報を検出してマスクするのに役立ちます。両方の機能を組み合わせることで、多層保護を提供できます。

この例では、メール、電話、名前、社会保障番号、銀行口座番号、クレジットカード番号を含むいくつかの管理データ識別子を使用して CloudWatch Logs データ保護ポリシーを作成します。また、データ保護ポリシーで使用できる独自のカスタム正規表現を定義できるカスタムデータ識別子（CDI）も含めました。カスタムデータ識別子を使用すると、管理データ識別子では提供できないビジネス固有の個人識別情報（PII）のユースケースを対象にできます。例えば、会社固有の従業員 ID を検索するためにカスタムデータ識別子を使用できます。カスタムデータ識別子は、管理データ識別子と併用できます。保護できるデータの種類の完全なリストについては、[ドキュメント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/protect-sensitive-log-data-types.html)を参照してください。


このエージェント Runtime ロググループのデータ保護ポリシーを有効にしましょう。オプションで、必要に応じて `aws/spans` ロググループのデータ保護ポリシーを有効にすることもできます。


In [ ]:
import json
cloudwatch_logs_client = boto3.client('logs')

log_group_name = '/aws/bedrock-agentcore/runtimes/' + launch_result.agent_id + '-DEFAULT'

data_protection_policy = {
    "Name": "data-protection-policy",
    "Description": "Policy to mask sensitive data in logs",
    "Version": "2021-06-01",
    "Statement": [
        {
              "Sid": "audit-policy",
              "DataIdentifier": [
                "arn:aws:dataprotection::aws:data-identifier/EmailAddress",
                "arn:aws:dataprotection::aws:data-identifier/PhoneNumber-US",
                "arn:aws:dataprotection::aws:data-identifier/Name",
                "arn:aws:dataprotection::aws:data-identifier/Ssn-US",
                "arn:aws:dataprotection::aws:data-identifier/BankAccountNumber-US",
                "arn:aws:dataprotection::aws:data-identifier/CreditCardNumber",
                "AgentId"
              ],
              "Operation": {
                "Audit": {
                  "FindingsDestination": {}
                }
              }
        },
        {
            "Sid": "redact-policy",
            "DataIdentifier": [
                "arn:aws:dataprotection::aws:data-identifier/EmailAddress",
                "arn:aws:dataprotection::aws:data-identifier/PhoneNumber-US",
                "arn:aws:dataprotection::aws:data-identifier/Name",
                "arn:aws:dataprotection::aws:data-identifier/Ssn-US",
                "arn:aws:dataprotection::aws:data-identifier/BankAccountNumber-US",
                "arn:aws:dataprotection::aws:data-identifier/CreditCardNumber",
                "AgentId"
            ],
            "Operation": {
                "Deidentify": {
                    "MaskConfig": {}
                }
            }
        }
    ],
    "Configuration": {
      "CustomDataIdentifier": [
        {
            "Name": "AgentId",
            "Regex": "[A-Z]{5}[0-9]{5}"
        }
      ]
    }
}

try:
    response = cloudwatch_logs_client.put_data_protection_policy(
        logGroupIdentifier=log_group_name,
        policyDocument=json.dumps(data_protection_policy)
        
    )
    print("データ保護ポリシーが正常に適用されました:")
    print(response)
except cloudwatch_logs_client.exceptions.ResourceNotFoundException:
    print(f"エラー: ロググループ '{log_group_name}' が見つかりません。")
except Exception as e:
    print(f"エラーが発生しました: {e}")


データ保護が有効になっていることを以下の方法で確認できます:

- CloudWatch コンソールにログイン
- ロググループ
- エージェントランタイムロググループを選択し（上記のレスポンスから 'logGroupIdentifier' で取得できます）、以下のように 'Data protection' タブを選択:


<div style="text-align:left">
    <img src="images/cloudwatch_logs_after_data_protection_enabled.png" width="100%"/>
</div>



では、ガードレールとデータ保護の両方を有効にしてエージェントを再度テストしましょう

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "summarize the agent conversation. Tell me exactly the phone number, name, address, and email?"})
invoke_response

ガードレールとログデータ保護の両方を有効にした結果を確認します。

以下のエージェントインタラクション（実際の出力は異なる場合がありますが、概念は同じです）:


<div style="text-align:left">
    <img src="images/agent_response_with_data_protection_enabled.png" width="100%"/>
</div>




トレース属性の機密情報も保護されていることがわかります。また、'agent.id' がマスクされていることにも注目してください。これは Regex を使用したカスタムデータ識別子に基づいています。


<div style="text-align:left">
    <img src="images/logs_with_data_protection_enabled.png" width="100%"/>
</div>





# 5. クリーンアップ（オプション）



In [ ]:
response = bedrock_client.delete_guardrail(
    guardrailIdentifier=guardrailId   # GUARDRAILID
)

In [ ]:
cloudwatch_response = cloudwatch_logs_client.delete_data_protection_policy(
    logGroupIdentifier=log_group_name
)

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)





# 6. まとめ

**主な学び**
おめでとうございます！エージェントの包括的なデータ保護対策を実装する方法を学びました。カバーした内容を振り返りましょう:

達成したこと

Bedrock Guardrails の設定:

- 機密情報の編集
- エージェントへのガードレールの適用

CloudWatch Logs データ保護の有効化:

- ログ内の機密データを自動的に検出してマスク
- 機密データおよびカスタムパターン用のデータ識別子の実装

Bedrock Agent Core との両方の機能のシームレスな統合による本番環境対応のデプロイメント


**ベストプラクティス**

- 防御を多層化: ガードレール（ランタイム保護）とログデータ保護（後処理セキュリティ）の両方を使用
- 徹底的にテスト: 本番デプロイ前に多様なテストケースでガードレールポリシーを検証
- 監視と改善: CloudWatch メトリクスと監査ログを定期的に確認して設定を改善
- 最小権限の原則: IAM ロールにガードレールとログに必要な権限のみを付与
- ポリシーの文書化: フィルタリングされるコンテンツとその理由の明確なドキュメントを維持


**次のステップ**
AI エージェントのセキュリティをさらに強化するには:

- 業界固有の用語のカスタム単語フィルターと正規表現パターンを探索
- 異なるガードレール設定で A/B テストを実装
- ガードレール介入メトリクスに CloudWatch アラームを設定
- 機密操作を含むロググループに AWS KMS 暗号化を検討


**追加リソース**

- [CloudWatch Logs データ保護監査結果](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/mask-sensitive-log-data-audit-findings.html)
- [アカウント全体のポリシー](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/mask-sensitive-log-data-accountlevel.html)
- [Bedrock Guardrails のユースケース](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use.html)


**注意**: 責任ある AI は一回限りの設定ではありません。継続的なコミットメントです。エージェントが進化し、新しい脅威が出現するにつれて、セーフガードを継続的に監視、評価、改善してください。

安全な構築を！